# Netz-Analyse

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

def shortest_path(G, start_point, end_point, weight='length'):
    '''
    Berechnet den Kürzesten Pfad von start_point zu end_point auf G mit dem Attribut weight
    '''
    # Kürzesten Pfad berechnen
    path = nx.shortest_path(G, start_point, end_point, weight=weight)
    
    # Ausgabe des Ergebnisses
    return path

def network_analysis(G, buildings, sources, weight='length'):
    '''Berechnet das Netz, indem der kürzeste Pfad zu jedem Gebäude gesucht wird'''

    buildings = buildings[buildings['HU_calc'] > 0]
    start_point = (sources['geometry'][0].x, sources['geometry'][0].y)
    
    # Neuer Graph für das Netzwerk erstellen
    net = nx.Graph()

    for row in buildings.itertuples():
        end_point = (row.centroid.x, row.centroid.y)
        
        # Die Leistung des Gebäudes extrahieren
        power = row.VOL
        
        path = shortest_path(G, start_point, end_point, weight)
        
        # Hinzufügen der Knoten und Kanten des Pfades zum Netzwerkgraphen
        for u, v in zip(path[:-1], path[1:]):
            # Kopiere alle Kantenattribute
            net.add_edge(u, v, **G.edges[u, v])
            # Aktualisiere die Leistung
            net.edges[u, v]['power'] = net.edges[u, v].get('power', 0) + power
    return net

def plot_network(G, net, buildings, sources):
    '''
    Zeigt das Straßennetzwerk G und das berechnete Netz net an.
    G: Straßennetzwerk-Graph
    net: Berechnetes Netz
    buildings: GeoDataFrame der Gebäude
    sources: GeoDataFrame der Energiequellen
    '''
    # Positionen der Knoten
    pos = {node: (node[0], node[1]) for node in G.nodes}

    # Figure und Axes erstellen
    fig, ax = plt.subplots(figsize=(20, 20))

    # Straßen zeichnen
    nx.draw_networkx_edges(G, pos=pos, ax=ax, edge_color='gray', alpha=0.7)

    # Kürzeste Pfade zeichnen
    for path in net.edges:
        nx.draw_networkx_edges(net, pos=pos, edgelist=[path], ax=ax, edge_color='blue', width=1.0)

    # Gebäude zeichnen
    buildings.plot(ax=ax, facecolor='#ff8888', edgecolor='black')

    # Energiequelle als Punkt zeichnen
    sources.plot(ax=ax, marker='o', markersize=15, color='green')

    # Raster und Achsentitel aktivieren
    #ax.grid(True)
    ax.set_title('Straßennetzwerk und berechnetes Netz')

    # Plot anzeigen
    plt.show()

net=network_analysis(G,buildings,sources)
#net2,i,cg=network_analysis2(G, net, buildings, sources)

plot_network(G,net,buildings,sources)
#plot_network(G,net2,buildings,sources)

# Graph zu shape

In [ ]:
from shapely.geometry import LineString

def ensure_power_attribute(graph):
    """
    Stellt sicher, dass jede Kante im Graphen das Attribut 'power' hat.
    Wenn eine Kante das Attribut nicht hat, wird es mit dem Wert 0 initialisiert.

    Parameters:
    - graph: Ein NetworkX-Graph

    Returns:
    - Der modifizierte Graph mit dem Attribut 'power' für jede Kante.
    """
    for u, v in graph.edges():
        if 'power' not in graph[u][v]:
            graph[u][v]['power'] = 0
    return graph

def graph_to_gdf(G):
    """Konvertiert einen networkx-Graphen in ein GeoDataFrame, wobei auch die Kantenattribute übernommen werden."""
    # Sammeln Sie die Kanten des Graphen und konvertieren Sie sie in LineString-Objekte
    # und sammeln Sie auch die Kantenattribute
    geometries = []
    attributes = {}

    for u, v, data in G.edges(data=True):
        geometries.append(LineString([u, v]))

        # Sammeln Sie Attribute für jede Kante
        for key, value in data.items():
            if key in attributes:
                attributes[key].append(value)
            else:
                attributes[key] = [value]

    # Erstellen Sie ein GeoDataFrame aus den LineString-Objekten und den Attributen
    gdf = gpd.GeoDataFrame(attributes, geometry=geometries)

    return gdf

g = ensure_power_attribute(net)
g = graph_to_gdf(g)

# GeoDataFrame als Shapefile
name = 'Netz'
g.to_file(f"{name}.shp")